1. Να διαβάζει τα δεδομένα από το αρχείο excel.

In [ ]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt

drive.mount('/content/gdrive')
df = pd.read_excel(r'/content/gdrive/My Drive/Dataset2Use_Assignment1.xlsx')

In [ ]:
df.head()

2. Να τυπώνει, σε γραφήματα τα ακόλουθα στοιχεία:

In [ ]:
# Αλλάζω τα στοιχεία της στήλης 'ΕΝΔΕΙΞΗ ΑΣΥΝΕΠΕΙΑΣ (=2) (ν+1)' από 1,2 σε 0,1
df['ΕΝΔΕΙΞΗ ΑΣΥΝΕΠΕΙΑΣ (=2) (ν+1)'] = df['ΕΝΔΕΙΞΗ ΑΣΥΝΕΠΕΙΑΣ (=2) (ν+1)'].replace({1: 1, 2: 0})

a. Figure 1: Αριθμό υγιών και χρεωκοπημένων επιχειρήσεων, για κάθε έτος.

In [ ]:
grouped = df.groupby(['ΕΤΟΣ', 'ΕΝΔΕΙΞΗ ΑΣΥΝΕΠΕΙΑΣ (=2) (ν+1)']).size().unstack()

# Plot the data with separate bars for healthy and broke companies
ax = grouped.plot(kind='bar', stacked=False)
plt.xlabel('Year')
plt.ylabel('Number of Companies')
plt.title('Number of Healthy and Broke Companies by Year')
plt.legend(['Broke', 'Good Financial Health'])

# Annotate the bars with the number of companies
for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy()
    ax.text(x+width/2,
            y+height/2,
            '{:.0f}'.format(height),
            horizontalalignment='center',
            verticalalignment='center')

b. Figure 2: Την min, max, average τιμή για κάθε δείκτη.


In [ ]:
# Separate the data into broke and healthy companies
broke_companies = df[df['ΕΝΔΕΙΞΗ ΑΣΥΝΕΠΕΙΑΣ (=2) (ν+1)'] == 0]
healthy_companies = df[df['ΕΝΔΕΙΞΗ ΑΣΥΝΕΠΕΙΑΣ (=2) (ν+1)'] == 1]

# Exclude the year and target columns
metrics = df.columns.drop(['ΕΤΟΣ', 'ΕΝΔΕΙΞΗ ΑΣΥΝΕΠΕΙΑΣ (=2) (ν+1)'])

# Calculate the min, max, and mean values for each metric for broke and healthy companies
broke_stats = broke_companies[metrics].agg(['min', 'max', 'mean'])
healthy_stats = healthy_companies[metrics].agg(['min', 'max', 'mean'])

# Create subfigures for broke and healthy companies
fig, ax = plt.subplots(2, 1, figsize=(15, 10))

# Create a bar plot for each metric for broke companies
broke_stats.T.plot(kind='bar', ax=ax[0], logy=True)
ax[0].set_title('Broke Companies')
ax[0].set_ylabel('Value (log scale)')
ax[0].tick_params(axis='x', rotation=90)

# Add the actual value on top of each bar
for p in ax[0].patches:
    ax[0].annotate(format(p.get_height(), '.2f'),
                   (p.get_x() + p.get_width() / 2., p.get_height()),
                   ha = 'center', va = 'center',
                   xytext = (0, 10),
                   textcoords = 'offset points')

# Create a bar plot for each metric for healthy companies
healthy_stats.T.plot(kind='bar', ax=ax[1], logy=True)
ax[1].set_title('Healthy Companies')
ax[1].set_ylabel('Value (log scale)')
ax[1].tick_params(axis='x', rotation=90)

# Add the actual value on top of each bar
for p in ax[1].patches:
    ax[1].annotate(format(p.get_height(), '.2f'),
                   (p.get_x() + p.get_width() / 2., p.get_height()),
                   ha = 'center', va = 'center',
                   xytext = (0, 10),
                   textcoords = 'offset points')

plt.tight_layout()
plt.show()

df.columns

3. Να ελέγχει για τυχόν ελλείπεις εγγραφές [π.χ. NaNs] και να ειδοποιεί τον χρήστη με σχετικό
μήνυμα.

In [ ]:
# Check for NaN values in the DataFrame
nan_values = df.isna()

# If there are any NaN values, inform the user with a message
if nan_values.any().any():
    print("There are NaN values in the following columns:")
    print(nan_values.any()[nan_values.any() == True])
else:
    print("There are no NaN values in the DataFrame.")

4. Να κανονικοποιεί τα δεδομένα στο διάστημα [0,1] με χρήση τεχνικής τύπου map minmax.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler to the data and transform the data
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

df_normalized.head()


5. Να κάνει χρήση του Stratified kfold ώστε να δημιουργεί 4 folds.

6. Να τυπώνει στην οθόνη, για κάθε fold, πόσες χρεωκοπημένες και πόσες υγιείς εταιρείες
υπάρχουν στο a) train και στο b) test set.

7. Αν η κατανομή είναι πάνω από 3 υγιείς επιχειρήσεις για κάθε χρεωκοπημένη, διαλέξτε με
τυχαίο τρόπο όσες υγιείς εταιρείες χρειαστεί, ώστε η αναλογία στο training set να είναι 3 υγιείς
/ 1 χρεωκοπημένη.

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Create a StratifiedKFold object
skf = StratifiedKFold(n_splits=4)

# Assuming that 'target' is the name of your target variable
X = df_normalized.drop('ΕΝΔΕΙΞΗ ΑΣΥΝΕΠΕΙΑΣ (=2) (ν+1)', axis=1)
y = df_normalized['ΕΝΔΕΙΞΗ ΑΣΥΝΕΠΕΙΑΣ (=2) (ν+1)']


import numpy as np

# Generate the stratified folds
fold = 1
folds = []
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Find the indices of the healthy and bankrupt companies in the training set
    healthy_indices = y_train[y_train == 1].index
    bankrupt_indices = y_train[y_train == 0].index

    # Calculate the number of healthy companies needed to achieve a 3:1 ratio
    required_healthy = len(bankrupt_indices) * 3

    if len(healthy_indices) > required_healthy:
        # Randomly select the required number of healthy companies
        selected_healthy_indices = np.random.choice(healthy_indices, size=required_healthy, replace=False)

        # Find the indices of the healthy companies that were not selected
        not_selected_healthy_indices = list(set(healthy_indices) - set(selected_healthy_indices))

        # Remove the not selected healthy companies from the training set and add them to the test set
        X_train = X_train.drop(not_selected_healthy_indices)
        y_train = y_train.drop(not_selected_healthy_indices)
        X_test = pd.concat([X_test, X.loc[not_selected_healthy_indices]])
        y_test = pd.concat([y_test, y.loc[not_selected_healthy_indices]])

    # Print the counts for the adjusted training set
    train_counts = y_train.value_counts()
    print(f"Fold {fold} Adjusted Training Set: {train_counts[0]} broke companies, {train_counts[1]} healthy companies")

    # Print the counts for the adjusted test set
    test_counts = y_test.value_counts()
    print(f"Fold {fold} Adjusted Test Set: {test_counts[0]} broke companies, {test_counts[1]} healthy companies")

    folds.append((X_train, y_train, X_test, y_test))

    fold += 1


8. Να υλοποιεί (εκπαιδεύει) τα ακόλουθα μοντέλα:
a. Linear Discriminant Analysis
b. Logistic Regression
c. Decision Trees
d. Random Forests
e. k-Nearest Neighbors
f. Naïve Bayes
g. Support Vector Machines
h. (Επιπλέον μοντέλο) Gradient Boosting Classifier

9. Για κάθε εκπαιδευμένο μοντέλο:
a. Να τυπώνει τα confusion matrices, ως figures, τόσο στο train όσο και στο test set.
b. Θα υπολογίζει την επίδοση τόσο στο train όσο και στο test set, με βάση τις ακόλουθες
μετρικές: Accuracy, Precision, Recall, F1 score, AUC ROC.
c. Να τυπώνει τα αποτελέσματα στην οθόνη με ακρίβεια 2 δεκαδικών ψηφίων.
d. Να καταχωρεί σε ένα dataframe (Pandas) σε μια νέα γραμμή τις ακόλουθες
πληροφορίες:
i. Classifier Name (str)
ii. Training or test set (str)
iii. Balanced or unbalanced train set (str)
iv. Number of training samples (int)
v. Number of non-healthy companies in training sample (int)
vi. True positives TP (int)
vii. True negatives TN (int)
viii. False positives FP (int)
ix. False negatives FN (int)
x. ROC-AUC (double)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import seaborn as sns

# Create the dataframe to store the results
results = pd.DataFrame(columns=['Classifier Name', 'Data Set', 'Train Set Balance', 'Number of Training Samples', 'Number of Non-Healthy Companies', 'TP', 'TN', 'FP', 'FN', 'ROC-AUC'])

# Create the models
models = {
    'Linear Discriminant Analysis': LinearDiscriminantAnalysis(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Trees': DecisionTreeClassifier(),
    'Random Forests': RandomForestClassifier(),
    'k-Nearest Neighbors': KNeighborsClassifier(),
    'Naïve Bayes': GaussianNB(),
    'Support Vector Machines': SVC(),
    'Gradient Boosting': GradientBoostingClassifier()
}

for name, model in models.items():
    for i, (X_train, y_train, X_test, y_test) in enumerate(folds, 1):
        # Train the model and make predictions
        model.fit(X_train, y_train)
        train_preds = model.predict(X_train)
        test_preds = model.predict(X_test)

    # Calculate the confusion matrices
    train_cm = confusion_matrix(y_train, train_preds)
    test_cm = confusion_matrix(y_test, test_preds)

    # Print the confusion matrices
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    sns.heatmap(train_cm, annot=True, fmt='d', ax=ax[0])
    ax[0].set_title(f'Train Confusion Matrix: {name} on Fold {i}')
    sns.heatmap(test_cm, annot=True, fmt='d', ax=ax[1])
    ax[1].set_title(f'Test Confusion Matrix: {name} on Fold {i}')
    plt.show()

    # Calculate the performance metrics
    for data, preds, true in [('Train', train_preds, y_train), ('Test', test_preds, y_test)]:
        accuracy = accuracy_score(true, preds)
        precision = precision_score(true, preds)
        recall = recall_score(true, preds)
        f1 = f1_score(true, preds)
        roc_auc = roc_auc_score(true, preds)

        # Print the results
        print(f'{name} on {data} Set of Fold {i}: Accuracy={accuracy:.2f}, Precision={precision:.2f}, Recall={recall:.2f}, F1 Score={f1:.2f}, ROC AUC={roc_auc:.2f}')

        # Add the results to the dataframe
        balance = 'Balanced' if len(y_train[y_train == 0]) == len(y_train[y_train == 1]) else 'Unbalanced'
        results = results.append({
            'Classifier Name': name,
            'Data Set': data,
            'Train Set Balance': balance,
            'Number of Training Samples': len(X_train),
            'Number of Non-Healthy Companies': len(y_train[y_train == 0]),
            'TP': confusion_matrix(true, preds)[1, 1],
            'TN': confusion_matrix(true, preds)[0, 0],
            'FP': confusion_matrix(true, preds)[0, 1],
            'FN': confusion_matrix(true, preds)[1, 0],
            'ROC-AUC': roc_auc
        }, ignore_index=True)

10. Όταν ολοκληρωθεί η εκτέλεση τα αποτελέσματα θα αποθηκεύουν σε ένα αρχείο csv με όνομα
balancedDataOutcomes.csv

In [ ]:
results.to_csv('balancedDataOutcomes.csv', index=False)

results = pd.read_csv('./balancedDataOutcomes.csv')

results.head()